<a href="https://colab.research.google.com/github/nikeyes/GenAI-experiments/blob/main/query-my-documents/LLM_queries_with_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import os
import requests
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('config.ini')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = parser.get('HuggingFace','ACCESS_TOKEN')

In [25]:
from langchain.document_loaders import UnstructuredURLLoader
urls = [
    "https://nikeyes.github.io/Principios-de-un-Open-Space-y-viajar-con-ni%C3%B1os-en-coche/",
    "https://nikeyes.github.io/cuando-cambiar-de-trabajo/"
]
loader = [UnstructuredURLLoader(urls=urls)]
#loader
doc = loader[0].load()


In [ ]:
#!pip install --upgrade pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)).from_loaders(loader)

In [27]:
## Text Splitter
#from langchain.text_splitter import CharacterTextSplitter
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
#docs = text_splitter.split_documents(doc)
## Embeddings
#from langchain.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings()#Create the vectorized db
## Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
#from langchain.vectorstores import FAISS
#db = FAISS.from_documents(docs, embeddings)

In [28]:
query = "Explícame la ley de los 2 pies con niños."
docs = db.similarity_search(query)
docs

[Document(page_content='1. Cualquier persona que se atreva a subir al coche con tus hijos es bien recibida.\n2. Lo que suceda es lo único que pudo suceder.\n3. Se arranca cuando se arranca.\n4. Se llega cuando se llega.\n\nY la ley de los dos pies:\n\nSi puedes usar tus dos pies y no subir al coche……\xa0 Hazlo insensato!!!\n\nFeliz viaje!\n\nWritten on July 16, 2018', metadata={'source': 'https://nikeyes.github.io/Principios-de-un-Open-Space-y-viajar-con-ni%C3%B1os-en-coche/'}),
 Document(page_content='Principios de un Open Space y viajar con niños en coche\n\nWritten on July 16, 2018\n\nTodo es divertido, mientras que le este pasando a otra persona.\n\nLa semana pasada tuve la suerte de poder asistir con la familia al Agile Open Space 2018 en Caldes de Montbui. \nhttps://aos2018.agile-spain.org/ \nEn el viaje de vuelta en coche con los niños, iba pensando en la similitud entre los principios de un Open Space y viajar con niños en coche. \nLos principios de un Open Space son:\n\n1. Cua

In [34]:
from langchain import HuggingFaceHub

#llm=HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large", model_kwargs={"temperature":0, "max_length":512})
#llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.1, "max_length":512})
llm=HuggingFaceHub(repo_id="mosaicml/mpt-7b", model_kwargs={"temperature":0.1, "max_length":512})


In [35]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [ ]:
#chain.run('Dime el tercer principio de un Open Space y viajar con niños en coche.')
chain.run('Explícame la ley de los 2 pies con niños.')

In [ ]:
ques = input('Your question: ')
chain.run(ques)

Your question: Dime el tercer principio de viajar con niños


ValueError: ignored